## Dashboard for CV furnace

This script publishes all the plots as an interative panel application. The Fastgrid template is used for dshboard creation. All the plots tested in the previous scripts are combined under this application.

Import required packages:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import holoviews as hv
import hvplot.pandas
import panel as pn
import data_processing_methods as dpm
from sklearn.gaussian_process import GaussianProcessRegressor
import plotly.graph_objects as go
import plotly.express as px
pn.extension('plotly')
import datetime as dt
from panel.template import DarkTheme

**dashboard for predicting O2 output**

In [28]:
# create widgets
text_box_1 = pn.widgets.TextInput(name='Temperature:', value='1100')
text_box_2 = pn.widgets.TextInput(name='Burner apporx. value:', value='0')
text_box_3 = pn.widgets.TextInput(name='Fan set point:', value='78')
text_box_4 = pn.widgets.TextInput(name='Damper range (high):', value='50')
text_box_5 = pn.widgets.TextInput(name='Damper range (low)', value='96')

# Read the dataframe
burner_settings_df = pd.read_excel('Burners_settings_vs_O2 _rev1.xlsx', sheet_name='Sheet1')
standardised_1_df = burner_settings_df.copy()
for col in burner_settings_df.columns:

    if (np.std(burner_settings_df[col])> 0):
        standardised_1_df[col] = dpm.standardise(burner_settings_df[col],np.mean(burner_settings_df[col]),np.std(burner_settings_df[col]))
    else:
        standardised_1_df = standardised_1_df.drop(columns=col)

X1 = standardised_1_df.drop(columns =['Output O2 / %', 'Output Burner usage / %'])
y1 = standardised_1_df['Output O2 / %']

# Drop columns with corr coeffcients higher than 0.9 
X1 = X1.drop(columns =['Burner turns from zero', 'Fan speed / RPM'])
X1_train = X1[10:]
y1_train = y1[10:]

# Train Gaussian Process regression model
gpr_1 = GaussianProcessRegressor().fit(X1_train, y1_train)
y1_std = np.std(burner_settings_df['Output O2 / %'])
y1_mean = np.mean(burner_settings_df['Output O2 / %'])

# define the plot functions for predicting O2 values
def plot_predictions():

    X1_predict = X1
    y1_predict = gpr_1.predict(X1_predict)
    y1_actual = y1
    Y1_df = pd.DataFrame(y1_predict*y1_std+y1_mean, columns=['Predicted'])
    Y1_df['Actual'] = y1_actual*y1_std+y1_mean 
    fig_a = Y1_df.hvplot(xlabel = 'Set point',ylabel = 'O2 output', height=400, width=1200, ylim =(0,10), grid= True, color=["#368dbc", "#8ec79a"])
    return fig_a

# define the plot functions for new user defined inputs
def plot_predictions_b(text_box_1,text_box_2,text_box_3,text_box_4,text_box_5):

    X_cols = list(X1_train.columns)
    X_new = pd.DataFrame([[int(text_box_1),int(text_box_2),int(text_box_3),int(text_box_4),int(text_box_5)]],
                         columns = X_cols)
    X_new.reset_index(drop=True, inplace=True)
    X_new_pr= X_new.copy()
    for col in X_cols:    
        X_new_pr[col] = dpm.standardise(X_new[col],np.mean(burner_settings_df[col]),np.std(burner_settings_df[col]))

    y_new_predict = gpr_1.predict(X_new_pr)
    Y_new = pd.DataFrame(y_new_predict*y1_std+y1_mean, columns=['Newly Predicted'])

    num = pn.indicators.Number(name='O2 Predictions', value=int(y_new_predict*y1_std+y1_mean), font_size ='40pt', format='{value}%', default_color ='white')

    return num

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# Output O2 predictions
""", width=500)

# Create interactive panels
layout_b_gague = pn.interact(plot_predictions_b, text_box_1 = text_box_1,text_box_2 = text_box_2,
                             text_box_3 = text_box_3,text_box_4 = text_box_4,text_box_5 = text_box_5)
layout_b_plot = pn.Row(layout_b_gague,plot_predictions)
layout_a = pn.Column(explanation_pane,layout_b_plot)

**Sensitivity analysis - scatter plots**

In [ ]:
# No. samples to be generated
N_samples = 5000
cols = list(X1_train.columns)

# Initialise arrays to store samples of un-standardised and 
# standardised inputs
D = len (cols)
X_samples_us = np.zeros([N_samples, D])
X_samples = np.zeros([N_samples, D])

for i in range(N_samples):
    for j in range (D):
        X_samples_us[i, j] = np.random.uniform(np.min(burner_settings_df[cols[j]]), np.max(burner_settings_df[cols[j]]))
        
# Standardise the samples created
for j in range (D):    
    X_samples[:,j] = dpm.standardise(X_samples_us[:,j],np.mean(burner_settings_df[cols[j]]),np.std(burner_settings_df[cols[j]]))

# Save the predictions 
X_samples_df = pd.DataFrame(data = X_samples, columns = cols)
X_samples_us_df = pd.DataFrame(data = X_samples_us, columns = cols)
y_samples_predict = gpr_1.predict(X_samples_df)
y_samples_predict = y_samples_predict*y1_std+y1_mean
X_samples_us_df['Predicted O2 %'] = y_samples_predict

cols_samples = list(X_samples_us_df.columns)
y_samples = pn.widgets.Select(name='Chose a signal', options=cols_samples ,value='Temperature setpoint / degC')

# Create widgets for setting input limits
temp_min_slider = pn.widgets.IntSlider(name='Temperature min limit', start=1100, end=1150, step=1, value = 1140)
temp_max_slider = pn.widgets.IntSlider(name='Temperature max limit', start=1100, end=1150, step=1, value = 1145)
O2_min_slider = pn.widgets.IntSlider(name='O2 min limit', start=0, end=10, step=1, value = 0)
O2_max_slider = pn.widgets.IntSlider(name='O2 maxt limit', start=0, end=10, step=1, value = 2)

@pn.depends(y_samples.param.value)
def plot_main_effects(y_samples,slider_pos_temp_min,slider_pos_temp_max,slider_pos_O2_min,slider_pos_O2_max):
    # Find the mean of each bin (binning data)
    n_bins = 30
    x_plot = X_samples_us_df[y_samples]
    y_plot = X_samples_us_df['Predicted O2 %']

    bins = np.linspace(np.min(x_plot), np.max(x_plot), n_bins)
    main_effect = np.zeros(len(bins)-1)
    main_effect_index = np.zeros(len(bins)-1)
    main_effect_df = pd.DataFrame({})

    
    for j in range(len(bins)-1):
        indx = np.logical_and(x_plot > bins[j], x_plot < bins[j+1])
        main_effect_index[j] = 0.5*(bins[j] + bins[j+1])

        # Only compute mean if there are any points in bin
        if np.sum(indx) > 0:
            main_effect[j] = np.mean(y_plot[indx])

    main_effect_df['index'] = main_effect_index
    main_effect_df['value'] = main_effect
    
    # plot horizontal and vertical lines for x-y limits  
    if (y_samples == 'Temperature setpoint / degC'):
        vline1 = hv.VLine(int(slider_pos_temp_min))
        vline1.opts(color= '#8ec79a')
        vline2 = hv.VLine(int(slider_pos_temp_max))
        vline2.opts(color= '#8ec79a')
        
        h1line = hv.HLine(int(slider_pos_O2_min))
        h1line.opts(color='#8ec79a')               
        h2line = hv.HLine(int(slider_pos_O2_max))
        h2line.opts(color='#8ec79a')

                
    # plot main effects of model inputs
    fig_c = X_samples_us_df.hvplot.scatter(x = y_samples, y = 'Predicted O2 %', height = 500, width = 1100, hover_cols = 'all', color="#368dbc", grid = True )

    if (y_samples == 'Temperature setpoint / degC'):
        fig = fig_c*vline1*vline2*h1line*h2line
    
        ind1 = np.logical_and(X_samples_us_df['Temperature setpoint / degC']>slider_pos_temp_min,X_samples_us_df['Temperature setpoint / degC']<slider_pos_temp_max)
        X_samples_a_df = X_samples_us_df[ind1]
        ind2 = np.logical_and(X_samples_a_df['Predicted O2 %']<slider_pos_O2_max,X_samples_a_df['Predicted O2 %']>slider_pos_O2_min)
        sorted_df = X_samples_a_df[ind2].sort_values(by=['Predicted O2 %'],ignore_index = True)

        num1 = pn.indicators.Number(name='Fan setpoint', value=int(sorted_df['Fan setpoint / %'][0]), font_size ='40pt', format='{value}%', default_color ='white')
        num2 = pn.indicators.Number(name='Burner apporx. value', value=int(sorted_df['Burner approx value'][0]), font_size ='40pt', format='{value}', default_color ='white')
        num3 = pn.indicators.Number(name='Damper range (high)', value=int(sorted_df[' damper range / % (low)'][0]), font_size ='40pt', format='{value}%', default_color ='white')
        num4 = pn.indicators.Number(name='Damper range (low)', value=int(sorted_df['damper range / % (High)'][0]), font_size ='40pt', format='{value}%', default_color ='white')
        num = pn.Column(num1,num2,num3,num4)
        fig_out = pn.Row(num,fig)
    else:
        fig_out = fig_c
    return fig_out

In [ ]:
# Plot  
explanation_pane = pn.pane.Markdown("""
# Main effects
""", width=500)

# Create interactive panels
widget = pn.Row(y_samples, pn.Column(temp_min_slider,temp_max_slider),pn.Column(O2_min_slider,O2_max_slider))
chart_interact = pn.bind(plot_main_effects, y_samples = y_samples, slider_pos_temp_min=temp_min_slider, slider_pos_temp_max=temp_max_slider, 
                             slider_pos_O2_min=O2_min_slider, slider_pos_O2_max=O2_max_slider)
layout_b = pn.Column(explanation_pane, widget,chart_interact)

**Dashboard tabs for visualising temp. data**

In [ ]:
# create widgets
df_merged = pd.read_pickle('merged_sensor_df.pkl')
df_merged = df_merged.resample('1T').mean()

cols_1 = list(df_merged.columns[df_merged.columns.str.startswith('ROOF_0104_300_')])
y1 = pn.widgets.Select(name='Chose a signal to tune filtering parameters', options=cols_1 ,value='ROOF_0104_300_10_TC')

slider = pn.widgets.FloatSlider(start=0.01, end=0.1, step =0.01, value=0.1, name='low pass filter (normalised cut-off freq.)')

dt_start_input = pn.widgets.DatetimeInput(name='Start date (choose a 4 day period for parameter tuning)', value=dt.datetime(2022, 9, 6))
dt_end_input = pn.widgets.DatetimeInput(name='End date', value=dt.datetime(2022, 10, 14))

# Define function for ploting sensor data
@pn.depends(y1.param.value)
def plot_trunc_furnace_data(date_start, date_end, y1,slider_pos,Remove_spikes = False):

    df_merged_truncated = df_merged.loc[:,df_merged.columns.str.startswith('ROOF_0104_300_')]
    df_merged_truncated = df_merged_truncated [date_start: date_end]
    fig_e = df_merged_truncated.hvplot(ylabel = 'Temp.(C)',height = 400, width = 1400, line_width = 3,grid= True,legend='left'
                                       ,color=["#368dbc", "#8ec79a","#d83569","#ec6608","#ffcc00","#6f3f81", "#c78a06", "#5d430a", "#fff69f", "#a15501"])
    df_merged_filtered = df_merged_truncated.copy()
    if Remove_spikes:
        df_merged_filtered.loc[:,y1] = dpm.remove_spikes(df_merged_truncated.loc[:,y1],olr_def=1)
    
    df_merged_filtered.loc[:,y1] = dpm.low_pass_filter(df_merged_filtered.loc[:,y1],wn=slider_pos)
    fig_f = df_merged_filtered.hvplot.line(y = y1, height = 400, width = 1400, color= 'black', label = 'Filtered signal',line_width = 2, ylim =(0,1200))
    return fig_e*fig_f

In [ ]:
# Plot  
explanation_pane = pn.pane.Markdown("""
# Roof temperatures
""", width=500)

# Create interactive panels
chart_b_interact = pn.interact(plot_trunc_furnace_data,date_start=dt_start_input, date_end=dt_end_input,y1=y1,slider_pos=slider)
layout_c = pn.Column(explanation_pane, chart_b_interact)

In [ ]:
df_merged = pd.read_pickle('merged_sensor_df.pkl')
df_merged = df_merged.resample('1T').mean()

# create widgets
cols_2 = list(df_merged.columns[df_merged.columns.str.startswith('PID_ZONE_1')])
y2 = pn.widgets.Select(name='Chose a signal to tune filter parameters', options=cols_2 ,value='PID_ZONE_1_PV')

slider = pn.widgets.FloatSlider(start=0.01, end=0.1, step =0.01, value=0.1, name='low pass filter (normalised cut-off freq.)')

dt_start_input = pn.widgets.DatetimeInput(name='Start date (choose a 4 day period for parameter tuning)', value=dt.datetime(2022, 9, 6))
dt_end_input = pn.widgets.DatetimeInput(name='End date', value=dt.datetime(2022, 10, 14))

# Define function for ploting PID zone 1 sensor data
@pn.depends(y2.param.value)
def plot_trunc_PID_data(date_start, date_end, y2,slider_pos,Remove_spikes = False):

    df_merged_truncated = df_merged.loc[:,df_merged.columns.str.startswith('PID_ZONE_1')]
    df_merged_truncated = df_merged_truncated [date_start: date_end]
    fig_g = df_merged_truncated.hvplot(ylabel = 'Temp.(C)',height = 400, width = 1400, line_width = 3,grid= True,legend='left',color=["#368dbc", "#8ec79a","#d83569","#ec6608","#ffcc00"])
    df_merged_filtered = df_merged_truncated.copy()
    if Remove_spikes:
        df_merged_filtered.loc[:,y2] = dpm.remove_spikes(df_merged_truncated.loc[:,y2],olr_def=1)
    
    df_merged_filtered.loc[:,y2] = dpm.low_pass_filter(df_merged_filtered.loc[:,y2],wn=slider_pos)
    fig_h = df_merged_filtered.hvplot.line(y = y2, height = 400, width = 1400, color= 'black', label = 'Filtered signal',line_width = 2, ylim =(0,1200))
    
    return fig_g*fig_h

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# PID Zone 1
""", width=500)

# Create interactive panels
chart_c_interact = pn.interact(plot_trunc_PID_data,date_start=dt_start_input, date_end=dt_end_input,y2=y2,slider_pos=slider)
layout_d = pn.Column(explanation_pane, chart_c_interact)

**Dispaly air flow**

In [ ]:
df_merged = pd.read_pickle('merged_sensor_df.pkl')
cols = list(df_merged.columns[df_merged.columns.str.startswith('AIR_')])

df_air_merged_selected = df_merged.loc[:, df_merged.columns.str.startswith('AIR_')]
df_air_merged_filtered = df_air_merged_selected.copy()

def plot_air_data():
      
    fig_k = df_air_merged_filtered.hvplot(height = 500, width = 1400, grid= True, ylim =(0,3000),color=["#368dbc", "#8ec79a","#d83569","#ec6608","#ffcc00","#6f3f81"])
    
    return fig_k

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# Air Flow
""", width=500)

# Create interactive panels with radio buttons and checkboxes
layout_f = pn.Column(explanation_pane, plot_air_data)

**Display gas flow**

In [ ]:
df_merged = pd.read_pickle('merged_sensor_df.pkl')
cols = list(df_merged.columns[df_merged.columns.str.startswith('GAS_')])

df_gas_merged_selected = df_merged.loc[:, df_merged.columns.str.startswith('GAS_')]
df_gas_merged_filtered = df_gas_merged_selected.copy()

def plot_gas_data():
       
    fig_m = df_gas_merged_selected.hvplot(height = 500, width = 1400, grid= True, color=["#368dbc", "#8ec79a","#d83569","#ec6608","#ffcc00","#6f3f81"])
    
    return fig_m

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# Gas Flow
""", width=500)

# Create interactive panels with radio buttons and checkboxes
layout_h = pn.Column(explanation_pane, plot_gas_data)

**Display O2 output**

In [ ]:
df_merged = pd.read_pickle('merged_sensor_df.pkl')
df_merged = df_merged.resample('1T').mean()

# create widgets
slider = pn.widgets.FloatSlider(start=0.01, end=0.1, step =0.01, value=0.1, name='low pass filter (normalised cut-off freq.)')

dt_start_input = pn.widgets.DatetimeInput(name='Start date (choose a 4 day period for parameter tuning)', value=dt.datetime(2022, 9, 6))
dt_end_input = pn.widgets.DatetimeInput(name='End date', value=dt.datetime(2022, 10, 14))

# Define function for ploting  O2 sensor data
def plot_O2_data(slider_pos, date_start, date_end, Remove_spikes = False):

    df_merged_truncated = df_merged.loc[:,df_merged.columns.str.startswith('FURNACE_0126_341_04_O2')]
    df_merged_truncated = df_merged_truncated [date_start: date_end]
    fig_i = df_merged_truncated.hvplot(ylabel = 'O2 output',height = 500, width = 1400, line_width = 5, grid= True,legend='left', color="#368dbc")
    df_merged_filtered = df_merged_truncated.copy()
    if Remove_spikes:
        df_merged_filtered.loc[:,'FURNACE_0126_341_04_O2'] = dpm.remove_spikes(df_merged_truncated.loc[:,'FURNACE_0126_341_04_O2'],olr_def=1)
    
    df_merged_filtered.loc[:,'FURNACE_0126_341_04_O2'] = dpm.low_pass_filter(df_merged_filtered.loc[:,'FURNACE_0126_341_04_O2'],wn=slider_pos)
    fig_j = df_merged_filtered.hvplot.line(y = 'FURNACE_0126_341_04_O2', height = 500, width = 1300, color= 'black', label = 'Filtered signal',line_width = 2)
    
    return fig_i*fig_j

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# Output O2
""", width=500)

# Create interactive panels
chart_b_interact = pn.interact(plot_O2_data,date_start=dt_start_input, date_end=dt_end_input,slider_pos=slider)
layout_e = pn.Column(explanation_pane, chart_b_interact)

**Predict furnace O2 output using air and gas flow**

In [ ]:
# Read all air flow sensor data
df_merged = pd.read_pickle('merged_sensor_df.pkl')
df_merged = df_merged.resample('1T').mean()
cols = list(df_merged.columns[df_merged.columns.str.startswith('AIR_')])

df_air_merged_selected = df_merged.loc[:, df_merged.columns.str.startswith('AIR_')]
df_air_merged_filtered = df_air_merged_selected.copy()

for col in cols:
    df_air_merged_filtered.loc[:,col] = dpm.remove_spikes(df_air_merged_selected.loc[:,col],olr_def=1)
    df_air_merged_filtered.loc[:,col] = dpm.low_pass_filter(df_air_merged_filtered.loc[:,col],wn=0.1)
    
# Read gas flow sensor data    
df_merged = pd.read_pickle('merged_sensor_df.pkl')
df_merged = df_merged.resample('1T').mean()
cols = list(df_merged.columns[df_merged.columns.str.startswith('GAS_')])

df_gas_merged_selected = df_merged.loc[:, df_merged.columns.str.startswith('GAS_')]
df_gas_merged_filtered = df_gas_merged_selected.copy()

for col in cols:
    df_gas_merged_filtered.loc[:,col] = dpm.remove_spikes(df_gas_merged_selected.loc[:,col],olr_def=1)
    df_gas_merged_filtered.loc[:,col] = dpm.low_pass_filter(df_gas_merged_filtered.loc[:,col],wn=0.1)

    # Read gas flow sensor data    
df_merged = pd.read_pickle('merged_sensor_df.pkl')
df_merged = df_merged.resample('1T').mean()
cols = list(df_merged.columns[df_merged.columns.str.startswith('FURNACE_0126_341_04_O2')])

df_O2_selected = df_merged.loc[:, df_merged.columns.str.startswith('FURNACE_0126_341_04_O2')]
df_O2_filtered = df_O2_selected.copy()

for col in cols:
    df_O2_filtered.loc[:,col] = dpm.remove_spikes(df_O2_selected.loc[:,col],olr_def=1)
    df_O2_filtered.loc[:,col] = dpm.low_pass_filter(df_O2_filtered.loc[:,col],wn=0.01)

df_flow_combined = pd.concat([df_air_merged_filtered, df_gas_merged_filtered], axis=1)
df_flow_combined = pd.concat([df_flow_combined,df_O2_filtered], axis=1)
df_flow_combined = df_flow_combined[30000:-100]


In [ ]:
# Standardise input data
standardised_df = df_flow_combined.copy()
for col in df_flow_combined.columns:

    if (np.std(df_flow_combined[col])> 0):
        standardised_df[col] = dpm.standardise(df_flow_combined[col],np.mean(df_flow_combined[col]),np.std(df_flow_combined[col]))
    else:
        standardised_df = standardised_df.drop(columns=col)

y = standardised_df['FURNACE_0126_341_04_O2']
X = standardised_df

In [ ]:
# Drop columns with corr coeffcients higher than 0.9 
X = X.drop(columns =['AIR_DRIVE_AIR_SPEED', 'AIR_0123_945_03_PT', 'GAS_0110_943_14_PT', 'GAS_0110_943_07_FT_m3_h'])

In [ ]:
def plot_standardised_data():
       
    fig_n = X.hvplot(height = 500, width = 1400, line_width = 2, grid= True, color=["#368dbc", "#8ec79a","#d83569","#ec6608","#ffcc00","#6f3f81", "#c78a06"])
    
    return fig_n

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# Air and gas flow 
##(filtered and normalised)
""", width=1000)

# Create interactive panels with radio buttons and checkboxes
layout_g = pn.Column(explanation_pane, plot_standardised_data)

In [ ]:
# Select the datafor training
X_train = X[0:int(len(X)/2)]
y_train = y[0:int(len(X)/2)]

# Train Gaussian Process regression model
gpr = GaussianProcessRegressor().fit(X_train, y_train)
y_std = np.std(df_flow_combined['FURNACE_0126_341_04_O2'])
y_mean = np.mean(df_flow_combined['FURNACE_0126_341_04_O2'])

In [ ]:
# Make predictions with model trained using gas and air flows
X_predict = X
y_predict = gpr.predict(X_predict)
y = y.reset_index()

In [ ]:
y_actual = y['FURNACE_0126_341_04_O2']
Y_df = pd.DataFrame(y_predict*y_std+y_mean, columns=['Predicted'])
Y_df['Actual'] = y_actual*y_std+y_mean 
Y_df['Date'] = y['Date']
Y_df.set_index('Date',inplace=True)

def plot_predicted_data():

    fig_o = Y_df.hvplot(xlabel = 'Date',ylabel = 'O2 output', height=500, width=1400, ylim =(-5,25), grid= True, color=["#368dbc", "#8ec79a"])            
    return fig_o

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# O2 predictions
##(w.r.t gas and air flows) 
""", width=1000)

# Create interactive panels with radio buttons and checkboxes
layout_i = pn.Column(explanation_pane, plot_predicted_data)

**Panel template**

In [ ]:
radio_buttons = pn.widgets.RadioButtonGroup(options=['O2 predictions - Burner settings', 'Temperature data visualisation','O2 output - Filtered','Air flow', 'Gas flow',
                                                     'Air and gas flow - Filtered','O2 predictions - Air and gas flow'], 
                                                    button_type = 'primary', orientation = 'vertical')

@pn.depends(radio_buttons)
def temp_type(x):
    if x == 'O2 predictions - Burner settings':
        temp_layout = pn.Column(layout_a,layout_b)
        return temp_layout
    if x == 'Temperature data visualisation':
        temp_layout = pn.Column(layout_c,layout_d)
        return temp_layout
    if x == 'O2 output - Filtered':
        temp_layout = layout_e
        return temp_layout
    if x == 'Air flow':
        temp_layout = layout_f
        return temp_layout
    if x == 'Gas flow':
        temp_layout = layout_h
        return temp_layout
    if x == 'Air and gas flow - Filtered':
        temp_layout = layout_g
        return temp_layout
    if x == 'O2 predictions - Air and gas flow':
        temp_layout = layout_i
        return temp_layout
        
temp_type = pn.bind(temp_type, x=radio_buttons)
template = pn.template.FastListTemplate(
            site="Dashboard", title="CV Furnace",
            sidebar = [radio_buttons], background_color = '#6f7271',
            header_background = '#3e4765', neutral_color = '#6f7271',
            accent_base_color = '#3e4765', header_accent_base_color = '#3e4765', 
            main=[temp_type],theme="dark", sidebar_width = 250
            ).servable()